# 円筒座標系のシュレーディンガー方程式のエルミート性
よく知られているように、ハミルトニアンはエルミートである。  
ハミルトニアンがエルミートである時、固有値は実数である。  
ここで、一つの疑問が生じる。
## 差分化して解いた行列はエルミート行列ではなかった
円筒座標系のシュレーディンガー方程式の動径方向に関する方程式は
$$
\left[ -\frac{\hbar^2}{2m} \left(\frac{\partial^2}{\partial r^2} + \frac{1}{r} \frac{\partial}{\partial r} \right) +\frac{\hbar^2 n^2}{2m r^2} + V(r) \right] \xi(r) = \epsilon \xi(r)
$$
である。以前のノートで、これを差分化して解いた。つまり、
一階微分を
$$
\frac{df}{dr}|_{r = r_i} = \frac{f(r_i+a) -f(r_i-a) }{2a}
$$
として、二階微分を
$$
\frac{d^2f}{dr^2}|_{r = r_i} = \frac{f(r_i+a) -2f(r_o)+ f(r_i-a)}{a^2}
$$
として、行列を作って、その固有値問題を解いた。以前の結果より、この固有値問題の固有値は実数である。  
しかし、ここで作った行列$H$はエルミート行列だろうか？
線形代数における、エルミート行列の定義は
$$
H^{\dagger} = H
$$
である。  
行列を以前と同じように定義すると

In [11]:
function calc_cij(i,j,a,N)
    cij = 0.0
    if i==1
        cij = ifelse(j==2,1.0,0.0)        
        cij += ifelse(j==1,-1.0,0.0)
    elseif i==N
        cij = ifelse(j==N-1,-1.0,0.0)
    else
        cij = ifelse(j==i+1,1.0,0.0)
        cij += ifelse(j==i-1,-1.0,0.0)
    end
    cij = cij/(2a)
    return cij
end
    
function calc_dij(i,j,a,N)
    dij = 0.0
    if i==1
        dij += ifelse(j==2,1.0,0.0)
        dij += ifelse(j==1,-1.0,0.0) #-2+1
    elseif i==N
        dij += ifelse(j==N,-2.0,0.0)        
        dij += ifelse(j==N-1,1.0,0.0)
    else
        dij += ifelse(j==i+1,1.0,0.0)
        dij += ifelse(j==i,-2.0,0.0)
        dij += ifelse(j==i-1,1.0,0.0)
    end
    dij = dij/(a^2)
    return dij
end

function make_Hr(a,N,n)
    mat_Hr = zeros(Float64,N,N)
    vec_V = zeros(Float64,N)
    for i in 1:N
        r = a*i -a/2
        mat_Hr[i,i] = (n^2/r^2 + vec_V[i])
        for dr in -1:1
            j = i + dr
            if 1 <= j <= N
                cij = calc_cij(i,j,a,N)
                dij = calc_dij(i,j,a,N)
                mat_Hr[i,j] += -(dij + cij/r)
            end
        end
    end
    return mat_Hr
end

make_Hr (generic function with 1 method)

である。この行列を用いて、
$$
H - H^{\dagger}
$$
を計算すると

In [12]:
a = 0.01
N = 10
n = 0
mat_Hr = make_Hr(a,N,n)

mat_sa = mat_Hr - mat_Hr'
for i in 1:N
    for j in 1:N
        if abs(mat_sa[i,j]) != 0.0
            println(i," ",j," ",mat_sa[i,j])
        end
    end
end

1 2 -13333.333333333334
2 1 13333.333333333334
2 3 -5333.333333333334
3 2 5333.333333333334
3 4 -3428.5714285714275
4 3 3428.5714285714275
4 5 -2539.682539682539
5 4 2539.682539682539
5 6 -2020.2020202020212
6 5 2020.2020202020212
6 7 -1678.3216783216794
7 6 1678.3216783216794
7 8 -1435.8974358974356
8 7 1435.8974358974356
8 9 -1254.901960784313
9 8 1254.901960784313
9 10 -1114.55108359133
10 9 1114.55108359133


となり、0にならない。
これは、一階微分が
$$
\frac{df}{dr}|_{r = r_i} = \frac{f(r_i+a) -f(r_i-a) }{2a}
$$
であるために、
$(i+1,i)$成分と$(i,i+1)=((i+1)-1,(i+1))$成分の符号が異なるため生じる。  
つまり、一階微分の項がある限り差分化した行列はエルミート行列にならない。 
なぜこのようなことが生じているのだろうか？

## ハミルトニアンがエルミートであることの証明
では、ハミルトニアン演算子がエルミートであることを言うにはどうすればいいのだろうか。  
ここで、エルミート演算子の定義を行う。ある関数$f({\bf x})$と$g({\bf x})$の内積を
\begin{equation}
(f,g) = \int d{\bf x} (f({\bf x}))^{\ast} g({\bf x})
\end{equation}
と定義すると、
と
エルミート演算子は
$$
(f,H g) = (H f,g)
$$
が成り立つ$H$のことを言う。
ここで、積分は二次元直交座標系であれば$\int dx \int dy$となる。  
一次元系で確かめてみよう。ポテンシャルの部分は可換であるため、二階微分の項だけを考える。  
内積は
\begin{equation}
(f,H g) = \int dx f^{\ast} \frac{d^2 g}{dx^2} = \int dx f^{\ast} \frac{d}{dx} \left(\frac{dg}{dx} \right)
\end{equation}
\begin{equation}
= \left[\frac{df^{\ast}}{dx}\frac{dg}{dx} \right]_{-\infty}^{\infty} - \int dx \frac{df^{\ast}}{dx}\frac{dg}{dx} = \left[\frac{df^{\ast}}{dx}g \right]_{-\infty}^{\infty} - \left[\frac{df^{\ast}}{dx} \frac{dg}{dx} \right]_{-\infty}^{\infty}+\int dx \frac{d^2f^{\ast}}{dx^2}g
\end{equation}
となり、無限遠方で関数の微分がゼロとなっていれば、
\begin{equation}
(f,H g) = (H g,f)
\end{equation}
が成り立つことになる。

次に、円筒座標系を考える。円筒座標系のハミルトニアンのうち、問題となる部分は
\begin{equation}
\left(\frac{\partial^2}{\partial r^2} + \frac{1}{r} \frac{\partial}{\partial r} \right) 
\end{equation}
であり、これは
\begin{equation}
\left(\frac{\partial^2}{\partial r^2} + \frac{1}{r} \frac{\partial}{\partial r} \right) = \frac{1}{r} \frac{\partial }{\partial r} \left(r \frac{\partial}{\partial r} \right)
\end{equation}
と書くことができる。  
そして、円筒座標系における内積だが、$\int dr d\theta$ではなく、$\int r dr d \theta$であることに注意しよう。  
関数$f$と$g$は$\theta$に依存しないことと、$f$と$g$が実数であるとすると、
内積は、
\begin{equation}
(f,H g) = \int_0^R r dr  f(r) \frac{1}{r} \frac{\partial }{\partial r} \left(r \frac{\partial}{\partial r} g(r)\right)
\end{equation}
である。  
部分積分を実行していくと、
\begin{equation}
(f,H g) = 
\left[ f(r) r \frac{\partial}{\partial r} g(r)\right]_0^R - \int_0^R \frac{\partial}{\partial r} f(r) r \frac{\partial}{\partial r} g(r)
\end{equation}
\begin{equation}
= \left[ f(r) r \frac{\partial}{\partial r} g(r)\right]_0^R - \left[ \frac{\partial}{\partial r} f(r) r g(r) \right]_0^R + \int_0^R \frac{\partial}{\partial r} \left( r \frac{\partial}{\partial r} f(r)\right)  g(r)
\end{equation}
となる。$r=0$で極端に変な関数でなく、$r=R$において、微分がゼロあるいは値がゼロであれば、
\begin{equation}
(f,H g) = (Hf,g)
\end{equation}
が得られ、ハミルトニアンはエルミート演算子であることを示すことができる。

## 内積の定義の変更
さて、以上の議論から、差分化した場合にエルミート行列にならない問題の解決が見えてきた。
行列の問題では、固有ベクトルの内積を求める際に、$r$をかけたりはしない。一方、本来は$r$をかけなければならない。ここに問題がある。
そこで、
$$
f \rightarrow r^{-1/2} f'(r)
$$
$$
g \rightarrow r^{-1/2} g'(r)
$$
という関数を用意してみる。この時、内積は
$$
(f,g) = \int r dr r^{-1/2} f'(r) r^{-1/2} g'(r) = \int dr f'(r) g'(r)
$$
となり、$f'$と$g'$の内積には$r$がつかない。つまり、この$f'$と$r'$に関するシュレーディンガー方程式を差分化すれば、エルミート行列が得られるはずである。

## シュレーディンガー方程式の変形
動径方向の波動関数を
$$
\xi(r) = r^{l} f(r)
$$
とおく。この時、一階微分の項は
$$
\frac{\partial}{\partial r} (r^l f(r)) = l r^{l-1} f(r) + r^l \frac{\partial}{\partial r} f(r)
$$
となり、二階微分の項は
$$
\frac{\partial^2}{\partial r^2} (r^l f(r)) = \frac{\partial}{\partial r} \left[l r^{l-1} f(r) + r^l \frac{\partial}{\partial r} f(r)\right] =l(l-1)r^{l-2}f(r)+ 2l r^{l-1}\frac{\partial}{\partial r} f(r)+r^l \frac{\partial^2}{\partial r^2} f(r) 
$$
となるので、シュレーディンガー方程式の微分の項は
$$
\left[\frac{\partial^2}{\partial r^2}+ \frac{1}{r} \frac{\partial}{\partial r} (r^l f(r)) \right]
(r^l f(r)) = r^l \frac{\partial^2}{\partial r^2} f(r) + (2l+1)r^{l-1} \frac{\partial}{\partial r} f(r)+l(l-1)r^{l-2}f(r)
$$
となる。  
ここで、一階微分の項が存在してしまうと、差分化した時にエルミート行列にならなかったことを思い出そう。  
つまり、
$$
l = -\frac{1}{2}
$$
とすれば、差分化してもエルミート行列になるはずである。  
そしてこれは、
$$
\xi(r) = r^{-1/2} f(r)
$$
とすることになるので、$\xi(r)$に関する内積に出てくる$r$は、ちょうど打ち消されることになる！  
以上より、動径方向のシュレーディンガー方程式は、無次元化した場合に、
$$
\left[ - \left(r^{-1/2} \frac{\partial^2}{\partial r^2}+\frac{3}{4}r^{-1/2-2} \right) +\frac{ n^2}{r^2}r^{-1/2} + V(r)r^{-1/2} \right] f(r) = \epsilon r^{-1/2} f(r)
$$
$$
\left[ -\frac{\partial^2}{\partial r^2}  +\frac{ n^2-3/4}{r^2} + V(r) \right] f(r) = \epsilon f(r)
$$
が得られる。  
ここで、境界条件をディスク領域とすると、$f(r=R)=0$となり、回転対称性から、$r=0$近傍での差分化も問題なく実行できる。  
ハミルトニアンは



In [13]:
function calc_dij(i,j,a,N)
    dij = 0.0
    if i==1
        dij += ifelse(j==2,1.0,0.0)
        dij += ifelse(j==1,-1.0,0.0) #-2+1
    elseif i==N
        dij += ifelse(j==N,-2.0,0.0)        
        dij += ifelse(j==N-1,1.0,0.0)
    else
        dij += ifelse(j==i+1,1.0,0.0)
        dij += ifelse(j==i,-2.0,0.0)
        dij += ifelse(j==i-1,1.0,0.0)
    end
    dij = dij/(a^2)
    return dij
end

function make_Hr2(a,N,n)
    mat_Hr = zeros(Float64,N,N)
    vec_V = zeros(Float64,N)
    for i in 1:N
        r = a*i -a/2
        mat_Hr[i,i] = ((n^2-3/4)/r^2 + vec_V[i])
        for dr in -1:1
            j = i + dr
            if 1 <= j <= N
                #cij = calc_cij(i,j,a,N)
                dij = calc_dij(i,j,a,N)
                mat_Hr[i,j] += -(dij )
            end
        end
    end
    return mat_Hr
end

make_Hr2 (generic function with 1 method)

となり、

In [14]:
a = 0.01
N = 10
n = 0
mat_Hr2 = make_Hr2(a,N,n)

mat_sa = mat_Hr2 - mat_Hr2'
for i in 1:N
    for j in 1:N
        if abs(mat_sa[i,j]) != 0.0
            println(i," ",j," ",mat_sa[i,j])
        end
    end
end

行列はエルミートとなる。  
固有値を比較してみよう。  
まず、$n=0$の時。

In [22]:
a = 0.01
N = 1000
n = 0
mat_Hr = make_Hr(a,N,n)

@time ε,ψ = eig(mat_Hr)
ε = sort(ε)

mat_Hr2 = make_Hr2(a,N,n)

@time ε2,ψ2 = eig(mat_Hr2)
println(ε[1:10])
println(ε2[1:10])

 39.840817 seconds (30 allocations: 15.551 MiB, 0.37% gc time)
  5.596800 seconds (30 allocations: 23.248 MiB, 0.12% gc time)
[0.057774, 0.304407, 0.748117, 1.389, 2.22705, 3.26228, 4.49467, 5.92421, 7.55088, 9.37468]
[-22716.1, -3.64628, 0.0688706, 0.387409, 0.934658, 1.70112, 2.68062, 3.86891, 5.26304, 6.86102]


結果は全然合わず、エルミート行列の方は変な負の固有値が出てきた。
これは、$n=0$の時、本来はベッセル関数
$$
J_0(r)
$$
が解であるが、これを
$$
J_0(r) = r^{-1/2}f(r)
$$
と分解しているために、計算で得るべき固有関数が
$$
f(r) = r^{1/2} J_0(r)
$$
という、原点で微分が発散するような鋭い変化をする関数になっているからである。その結果、差分化でこの関数がうまく記述できず、変な固有値が出てきてしまう。  
では、$n=1$ではどうだろうか？

In [15]:
a = 0.01
N = 1000
n = 1
mat_Hr = make_Hr(a,N,n)

@time ε,ψ = eig(mat_Hr)
ε = sort(ε)

mat_Hr2 = make_Hr2(a,N,n)

@time ε2,ψ2 = eig(mat_Hr2)
println(ε[1:10])
println(ε2[1:10])

 45.947976 seconds (30 allocations: 15.551 MiB, 0.01% gc time)
  7.285813 seconds (30 allocations: 23.248 MiB, 0.09% gc time)
[0.146673, 0.491691, 1.03395, 1.77341, 2.71005, 3.84386, 5.17484, 6.70296, 8.42821, 10.3506]
[0.117636, 0.433741, 0.947023, 1.65743, 2.56496, 3.66957, 4.97126, 6.47001, 8.16579, 10.0586]


となり、似た結果が得られる。計算速度は、エルミート行列の方が圧倒的に速い。  
どちらが精度が良いかを確認する。  
ポテンシャルがない時は、ベッセル関数のゼロ点を使って固有値を表すことができるので、

In [20]:
bessj1zero=[3.83171,7.01559,10.1735,13.3237,16.4706,19.6159,22.7601,25.9037,29.0468,32.1897,35.3323,38.4748,41.6171,44.7593,
47.9015,51.0435,54.1856,57.3275,60.4695,63.6114,66.7532,69.8951,73.0369,76.1787,79.3205,82.4623,85.604,88.7458,
91.8875,95.0292,98.171,101.313,104.454,107.596,110.738,113.879,117.021,120.163,123.304,126.446,129.588,132.729,
135.871,139.013,142.154,145.296,148.438,151.579,154.721,157.863,161.004,164.146,167.288,170.429,173.571,176.712,
179.854,182.996,186.137,189.279,192.421,195.562,198.704,201.845,204.987,208.129,211.27,214.412,217.554,220.695,
223.837,226.978,230.12,233.262,236.403,239.545,242.686,245.828,248.97,252.111,255.253,258.395,261.536,264.678,
267.819,270.961,274.103,277.244,280.386,283.527,286.669,289.811,292.952,296.094,299.235,302.377,305.519,308.66,
311.802,314.943]
R = 10.0
aε = []
for i in 1:10
    push!(aε,bessj1zero[i]^2/R^2)
end
println(aε)

ind = []
for i in 1:10
    push!(ind,i)
end

using Plots
gr()
plot(ind,[ε[1:10],ε2[1:10],aε[1:10]],label=["Non hermitian matrix","hermitian matrix","Exact results"],marker=:circle)

Any[0.14682, 0.492185, 1.035, 1.77521, 2.71281, 3.84784, 5.18022, 6.71002, 8.43717, 10.3618]


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Non hermitian matrix 
 
 
 
 
 
 hermitian matrix 
 
 
 
 
 
 Exact results

となり、エルミート行列のだけが精度が悪い。もしかするとどこかにバグがあるのかもしれない。
あるいは、$r=0$で発散する$r^{-1/2}$という関数の性質が悪いために、精度が悪いのかもしれない。

In [21]:
N =3000
R = 10
a = R/N
n = 1
mat_Hr2 = make_Hr2(a,N,n)

@time ε2,ψ2 = eig(mat_Hr2)

 39.208204 seconds (33 allocations: 207.071 MiB, 0.68% gc time)


([0.117726, 0.434087, 0.947801, 1.65883, 2.56718, 3.67283, 4.97579, 6.47605, 8.17361, 10.0685  …  3.59991e5, 3.59992e5, 359994.0, 3.59995e5, 3.59997e5, 3.59998e5, 3.59999e5, 3.59999e5, 3.6e5, 3.6e5], [6.75416e-6 -1.47957e-5 … -0.000439782 0.000274553; 1.35083e-5 -2.95913e-5 … 0.000879562 -0.000549105; … ; 5.90552e-5 0.000113399 … 9.72656e-5 4.34839e-5; 2.95276e-5 5.66996e-5 … -4.86329e-5 -2.17419e-5])

刻みの数を増やしても精度があまり改善されないことから、何らかのバグ、あるいは境界条件などに問題があるのかもしれない（調査中）。

## 現時点の結論：エルミート行列を作れるが、精度が良いとは限らない